In [1]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.

import pandas as pd
# Make the graphs a bit prettier, and bigger
pd.set_option('display.mpl_style', 'default')

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 20)

import numpy as np
import math
# The usual preamble
%matplotlib inline
%pylab inline
import matplotlib.pyplot as plt
plt.rcParams['axes.color_cycle'] = ['r', 'g', 'b', 'c']
plt.rcParams['lines.color'] = 'r'
plt.rcParams['figure.figsize'] = (15, 5)

import process
from modeling import categorize

Populating the interactive namespace from numpy and matplotlib


/Users/along528/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
import pickle
#clf = pickle.load(open('randomforestclassifier.p','rb'))
best_feature_names = pickle.load(open('best_features_names.p','rb'))
model = pickle.load(open('randomforestclassifier_bad.p','rb'))
print model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=15, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [3]:
df_with_rpsi = process.add_features(process.get_data(munge=True,with_traffic=True))
#df_all = process.add_features(process.get_data(munge=False,with_traffic=False))
df_all = process.add_features(process.get_data(munge=True,with_traffic=False,drop_features=False))
df_all_drop = process.add_features(process.get_data(munge=True,with_traffic=False))

#df_all = df_all_munge.merge(df_all,how='left') #in case dropped some during munging
df_all = df_all.merge(pd.DataFrame(df_with_rpsi['rpsi']),how='left',right_index=True,left_index=True)
df_all['is_measured'] = np.isnan(df_all['rpsi'])==False


/Users/along528/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [4]:
val_df,_ = process.get_split_add_data()
processor = process.Processor(val_df[best_feature_names])
X_predict,_ = processor.get_scaled_Xy(df_all_drop[best_feature_names])
y_predict = model.predict(X_predict)
df_all_predictions = pd.DataFrame(y_predict,columns=['rpsi_predicted'],index=df_all_drop.index)
df_all = df_all.merge(df_all_predictions,how='left',right_index=True,left_index=True)
df_all = df_all.fillna(-1)
df_all = df_all[((df_all['rpsi']==-1) & (df_all['rpsi_predicted']==-1))==False]
df_all

In [5]:
df_all.to_sql('app_db',process.engine,if_exists='replace')

In [6]:
df_all.to_csv('app_db.csv')

In [7]:
df_all

,agency,city,state,zipcode,agcytype,swnauthemp,swnftemp,swnptemp,civftemp,civptemp,totftemp,totptemp,ftreserveswn,ptreserveswn,ftreserveciv,ptreserveciv,ftgangoff,ptgangoff,ftdrugoff,ptdrugoff,ftterroff,pterroff,fthumtrfoff,pthumtrfoff,numrespoff,numcpo,numsro,numpatr,numinvst,numjail,...,sgtmax_per_capita,entrymin_per_capita,entrymax_per_capita,nummrkcars_per_capita,numothmrk_per_capita,numumkcars_per_capita,numothunm_per_capita,numplanes_per_capita,numcopters_per_capita,numboats_per_capita,nummotor_per_capita,numcarcam_per_capita,numfixcam_per_capita,nummobcam_per_capita,population_per_capita,black_over_white_population_disparity,black_over_white_income_disparity,black_over_white_institutionalized_disparity,black_over_white_institutionalized_adult_disparity,black_over_white_institutionalized_adult_federal_detention_disparity,black_over_white_institutionalized_adult_federal_prison_disparity,black_over_white_institutionalized_adult_state_prison_disparity,black_over_white_institutionalized_adult_local_jail_disparity,black_over_white_institutionalized_juvenile_disparity,black_officer_disparity,white_officer_disparity,black_over_white_officer_disparity,rpsi,is_measured,rpsi_predicted
surveyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Anchorage Police Department,Anchorage,AK,99507,3,399,385,0,174,0,559,0,0,0,0,0,6,0,12,0,6,0,7,0,182,6,18,182,76,0,...,92000.0,55000.00,95000.0,169,27,104,19,0,0,0,12,15,0,0,284142,0.092703,0.087302,0.131329,0.228111,1.000000,1.000000,1.000000,0.252708,0.148649,0.940382,1.345001,0.699168,-1,False,1
1001,Ketchikan Police Department,Ketchikan,AK,99901,3,23,23,0,10,1,33,1,0,0,0,0,0,0,0,0,0,0,0,0,23,0,2,0,0,0,...,73819.0,45635.00,59550.0,8,4,4,0,0,0,0,0,14,9,0,7384,0.008583,0.005714,0.070175,0.000000,1.000000,1.000000,0.000000,1.000000,0.214286,0.000000,1.148574,0.000000,-1,False,0
16,Tuscaloosa Police Department,Tuscaloosa,AL,35401,3,269,264,0,75,0,339,0,0,0,0,0,0,0,11,0,1,0,0,0,157,7,10,157,58,0,...,58445.0,38475.00,51054.0,116,18,71,13,0,3,4,8,38,48,1,83811,0.722283,0.688996,1.017279,1.487805,1.000000,1.000000,1.000000,1.487805,0.783784,0.369151,1.517748,0.243223,-1,False,1
13,Mobile Police Department,Mobile,AL,36606,3,532,526,0,254,96,780,96,0,6,0,0,0,0,5,0,1,0,0,0,304,30,5,345,97,0,...,52692.0,26616.00,41280.0,237,26,132,37,0,0,1,16,100,40,0,253842,0.887893,0.763822,0.922970,1.873085,1.000000,1.000000,1.000000,1.873085,2.195122,0.622467,1.391109,0.447461,-1,False,1
1004,Anniston Police Department,Anniston,AL,36202,3,92,81,0,33,0,114,0,0,0,0,0,0,0,1,0,0,0,0,0,69,0,0,0,0,0,...,51975.0,29310.00,43968.0,67,4,16,4,0,0,0,1,40,6,0,23736,0.392830,0.413985,0.371879,0.723881,1.000000,1.000000,1.000000,0.723881,0.551020,0.230880,1.360448,0.169709,-1,False,1
5,Birmingham Police Department,Birmingham,AL,35203,3,914,768,0,400,0,1168,0,0,0,0,0,4,0,0,0,1,0,0,0,462,462,12,528,137,1,...,54724.0,30451.00,47174.0,346,38,192,54,0,0,0,23,65,0,0,227686,0.872831,0.813041,0.856183,1.704591,1.000000,1.000000,0.618497,2.277439,2.373333,1.421503,0.735633,1.932355,-1,False,1
15,Montgomery Police Department,Montgomery,AL,36101,3,510,491,0,97,0,588,0,0,0,0,0,2,0,1,0,2,0,0,0,212,12,15,212,88,2,...,58870.0,32014.00,45573.0,216,34,133,25,0,0,0,15,89,0,13,202062,1.488363,1.322901,1.429854,1.529372,5.372549,1.235294,1.173724,4.019802,1.941748,0.723585,1.518510,0.476510,-1,False,1
7,Dothan Police Department,Dothan,AL,36303,3,157,148,0,100,0,248,0,0,0,0,15,0,0,0,0,0,0,0,0,80,0,12,80,38,22,...,59000.0,30800.00,47200.0,100,9,40,6,0,0,0,2,100,0,0,64931,0.434323,0.381148,0.665714,1.319328,1.000000,1.000000,1.000000,1.335106,1.190476,0.256933,1.359392,0.189006,-1,False,2
1018,Frisco City Police Department,Frisco City,AL,36445,3,4,3,2,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,...,999999.0,24960.00,42960.0,1,0,3,0,0,0,0,0,0,0,0,1417,0.375639,0.390131,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.272361,0.955896,1.331066,-1,False,0


In [8]:
df_all[df_all['agency'].str.contains('raleigh'.title())]

,agency,city,state,zipcode,agcytype,swnauthemp,swnftemp,swnptemp,civftemp,civptemp,totftemp,totptemp,ftreserveswn,ptreserveswn,ftreserveciv,ptreserveciv,ftgangoff,ptgangoff,ftdrugoff,ptdrugoff,ftterroff,pterroff,fthumtrfoff,pthumtrfoff,numrespoff,numcpo,numsro,numpatr,numinvst,numjail,...,sgtmax_per_capita,entrymin_per_capita,entrymax_per_capita,nummrkcars_per_capita,numothmrk_per_capita,numumkcars_per_capita,numothunm_per_capita,numplanes_per_capita,numcopters_per_capita,numboats_per_capita,nummotor_per_capita,numcarcam_per_capita,numfixcam_per_capita,nummobcam_per_capita,population_per_capita,black_over_white_population_disparity,black_over_white_income_disparity,black_over_white_institutionalized_disparity,black_over_white_institutionalized_adult_disparity,black_over_white_institutionalized_adult_federal_detention_disparity,black_over_white_institutionalized_adult_federal_prison_disparity,black_over_white_institutionalized_adult_state_prison_disparity,black_over_white_institutionalized_adult_local_jail_disparity,black_over_white_institutionalized_juvenile_disparity,black_officer_disparity,white_officer_disparity,black_over_white_officer_disparity,rpsi,is_measured,rpsi_predicted
surveyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
594,Raleigh Police Dept,Raleigh,NC,27602,3,764,710,0,114,0,824,0,0,0,0,0,1,0,1,0,2,0,0,0,305,6,10,305,110,0,...,72540.33,32649.13,55544.12,320,36,178,50,0,0,0,21,70,0,0,367120,0.422356,0.378851,0.985024,1.365526,1,1,1.14524,1.757396,3.761905,0.547183,1.317992,0.415164,4.805881,True,1
